In [9]:
import pandas as pd
import os
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Alignment, Border, Side, Font, PatternFill
from datetime import datetime
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import win32com.client as win32
pd.set_option('mode.chained_assignment', None)
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

# Function to send emails
def send_email(to_email, cc_email, subject, message):
    # Your email configuration for Outlook
    outlook = win32.Dispatch('Outlook.Application')
    mail = outlook.CreateItem(0)
    mail.Subject = subject
    mail.HTMLBody = message
    mail.To = to_email
    mail.CC = cc_email
    
    # Send the email
    mail.Send()
    
# Get the current date dynamically
current_date = datetime.now().strftime("%d-%m-%Y")
csv_file_path = f'Mis_report_{current_date}.csv'

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file_path)

# Load the data from the CSV file with 'latin-1' encoding
remark_df = pd.read_csv('C:/Users/Sahil Chipkar/Downloads/Python Scripts/No Information/Remark FIles/Remark.csv', encoding='latin-1')

# Merge the 'remark_df' with the original 'df' using the 'AWB' column as the key
df = df.merge(remark_df, on='AWB', how='left')

# Drop the specified columns
columns_to_remove = ['SrNo','Mode', 'NOP', 'AW', 'CW', 'Consignor Invoice No', 'Consignor Invoice Value', 'Bill Type',
                     'TAT', 'EDD Date', 'RTO Date', 'RTO Reason', 'Franchise Code', 'Franchise Name',
                     'Master Franchise Name', 'eWay no', 'Eway Expiry date', 'Pickupinscan date & time',
                     'pickupinscan branch', 'Booking Branch Out scan Date & Time', 'Deliverd TO',
                     'Delivery branch In-scan Date & Time', 'DRS Date & time', 'Regular/ODA', 'PRQ No',
                     'Pickup Genarte Date', 'Pickup Requested Date', 'PRQ Closed Date', 'PRQ Comment',
                     'Cash Invoice No', 'Invoice Date', 'PayBy', 'Payment Ref No', 'Coloader Name', 'CD No',
                     'CD Outscan', 'CD Inscan', 'Product Desc', 'Type Of Package','POD Uploaded Date & Time','Delivery Date','Sales Person Name','Sales Person Branch']
df = df.drop(columns=columns_to_remove, axis=1)

values_to_remove = [
    500126061, 500126062, 100019489, 100021439, 100018608, 500124003, 500124005, 500124008, 500124012,
    100023996, 100023034, 100011762, 100014746, 100010839, 100004010, 100004007, 100004009, 100004015,
    100004018, 100004084, 100004086, 100004088, 100004089, 100008470, 100008652, 100008817, 100008818,
    100008820, 'FBI122826', 100014745, 100014747, 100014750, 100014751, 1000014752, 100014454, 100014860,
    100014859, 100014462, 50119422, 100016765, 100014475, 100014478, 100018602, 100014748, 100016763,
    100016762, 100008471, 100008474, 100008475, 100025766, 100025772, 100025775, 100025788, 100025933,
    100002573, 100008470, 100017166, 100019553, 100019554, 100019555, 100019556, 100019557, 100019558,
    100019559, 100019560, 100019601, 100019602, 100019603, 100019540, 100019541, 100019542, 100019543,
    100011832, 100022473, 100022474, 100022475, 100022476, 100022477, 100022478, 100022479, 100022480,
    100022481, 100022483, 100022484, 100022485, 100022486, 100011817, 500124469, 100018908, 100018909,
    100018911, 100018912, 100018913, 100018914, 100018915, 100018916, 100018917, 100018831, 100018832,
    100018833, 100018834, 100018835, 100018836, 100018837, 100018838, 100018839, 100018890, 100018891,
    100018892, 100018893, 100019396, 100019397, 100019398, 100019399, 100019400, 100019401, 100019402,
    100018798, 100018799, 100018800, 100018801, 100018737, 100018738, 100018739, 100014754, 50113082,
    50113085, 50113086, 50113088, 50113091, 50113095, 50113281, 50113282, 50113285, 50113376,
    'FBI305510', 500126177, 100024107, 100024108, 100024111, 500125460, 500125064, 500124922, 100024045,
    100023775, 500124015, 500124031, 500124036, 'FBI122336', 100008729, 100008731, 'FBI121307', 'FBI120198',
    'MFBI112415', 500126147, 100024109,100014867, 100024110, 100024112, 100025669, 100009555, 100009556, 100018239,100023873,100008655,100014855100014854,100014857,100014858]

# Remove the specified values from the DataFrame
df = df[~df['AWB'].astype(str).isin(map(str, values_to_remove))]

# Add a new column for aging based on Booking Date and Today's Date
df['Last scan Date & time'] = pd.to_datetime(df['Last scan Date & time'])  # Convert to datetime, format='%d/%m/%Y %H:%M'
df['Ageing'] = (datetime.now() - df['Last scan Date & time']).dt.days

# Reorder the columns
column_names = ['Booking Date', 'AWB','Ageing','Booking Branch', 'Destination', 'Customer code',
                'Customer Name', 'Consignor Origin', 'Consignor', 'Consignee', 'Consignee Pincode',
                'Current Status', 'Last Scan Branch', 'Last scan Date & time', 'POD Status', 'DRS Branch','Remark','Change']

df = df[column_names]
df1= df
df2 = df
# Define the list of allowed 'Current Status' values for Destination filter  #'In transit',
allowed_statuses_destination = ['Out For Delivery','Franchise Menifiest In-Scan','In transit']

# Define the list of allowed 'Current Status' values for Booking Branch filter
allowed_statuses_booking_branch = ['Booked', 'In-Scan-Branch', 'In Scan Ahmedabad', 'In Scan Delhi', 'Pickup-In-scan', 'In Scan Hyderabad']

# Define the new 'Current Status' values for the additional filter
allowed_statuses_last_scan = ['Bag In-Scan', 'Bag genrated', 'Bag generated', 'Menifiest In-Scan', 'Manifest genrated', 'Master Manifest in-scan']

# Define branch data for each city (for Destination filter)
branch_origin_destination = [
    ('Hyderabad', ['KALYANDURG',"KURNOOL", "RAMCHANDRAPURAM", "EAST GODAVARI", "HYDERABAD", "GUNTUR","KRISHNA", "ANANTAPUR", "VIJAYAWADA", "NELLORE", "SRIKAKULAM","KOTHAGUDEM", "VISHAKAPATNAM", "SECUNDRABAD", "WARANGAL", "MEDAK","VIZIANAGARAM", "CUDDAPAH", "ADILABAD", "NIZAMABAD", "KARIM NAGAR","MAHABUB NAGAR", "KHAMMAM", "NALGONDA", "KAKINADA", "WEST GODAVARI","ELURU", "K.V.RANGAREDDY", "TIRUPATI", "CHOTUPPAL", "K V Rangareddy","CHITTOOR", "RAJAMUNDHRY", "PRAKASAM", "TANUKU", "BHIMAVARAM","RAMACHANDRAPURAM", "HATHNOOR", "PATTANCHERUVU"] ),
    ('Bangalore',['MYSORE','AMBEDKAR NAGAR','TIRUVALLUR','MALAPURAM','TRICHY','HAVERI','SALEM','PATHANAMTHITTA',"BANGALORE","MANGALORE", "TUMKUR", "Nelamangala", "DHARWAD", "GULBARGA","SHIMOGA", "BELLARY", "KOLAR", "KARWAR", "DAVANAGERE","ERNAKULAM", "HAVERI", "BELGAUM", "BAGALKOT", "KOZHIKODE","RAICHUR", "BIDAR", "BIJAPUR", "HINDUPUR", "CHITRADURGA","RAMANGARA", "UDUPI", "THRISSUR", "HASSAN", "MANDYA","MALAPPURAM", "KOLLAM", "Kannur", "KASARGOD","THIRUVANANTHAPURAM", "CALICUT", "PALAKKAD", "YADGIR","KODAGU", "GADAG", "SIRSI", "KOTTAYAM", "Bangalore Rural","COCHIN", "HUBLI", "ALWAYE", "TRICHUR", "ALLEPPEY","CANNANORE", "DAKSHINA KANNADA", "QUILON", "HOSKOTE","CHIKKABALLAPUR", "DODBALAPUR"] ),
    ('Kolkata', ['BARDHAMAN','KAMRUP','HOOGHLY','SILIGURI','BARGARH','CUTTACK',"JAMSHEDPUR", "RANCHI", "PATNA", "KOLKATA", "HOWRAH", "BHUBANESWAR", "BHAGALPUR", "HOOGHLY", "BEGUSARAI", "SILIGURI", "KAMRUP", "BHADRAK", "GUWAHATI", "DEOGHAR", "BALANGIR", "BARDHAMAN", "BERHAMPUR", "KISHANGANJ", "EAST CHAMPARAN", "NAYAGARH", "JHARSUGUDA", "VAISHALI", "KHURDA", "WEST MIDNAPORE", "JAJAPUR", "SUNDERGARH", "BALASORE", "GAYA", "CUTTACK", "DHANBAD", "NADIA", "DARBHANGA", "SAMBALPUR", "BARGARH", "DHENKANAL", "AGARTALA", "BHOJPUR", "ANGUL", "PURI", "JORHAT", "SOUTH 24 PARGANAS", "MUZAFFAPUR", "ITANAGAR", "BOKARO", "NALANDA", "SIWAN", "RAYAGADA", "PARADEEP", "GANGARAMPUR", "CACHAR", "KENDRAPARA", "NORTH 24 PARGANAS", "MADHUBANI", "KODERMA", "IMPHAL", "GOPALGANJ", "BURDWAN", "BARPETA", "KOKRAJHAR", "MALDA", "DABGRAM", "ROURKELA", "MOTIHARI", "KHORDA", "HALDIA", "SILCHAR", "NAHARKATIA", "ASANSOL", "MAYURBHANJ"]),
    ('Chennai', ['Tiruchirappalli','KANYAKUMARI','HOSUR', 'GUMMIDIPOONDI', 'Tirumudivakkam', 'TIRUVALLUR', 'Padappai', 'CHENGALPATTU', 'CHENNAI', 'COIMBATORE', 'DINDIGUL', 'ERODE', 'KANCHIPURAM', 'KRISHNAGIRI', 'MADURAI', 'PADAPPAI', 'PONDICHERRY', 'SALEM', 'TIRUNELVELI', 'TRICHY', 'VELLORE']),
    ('Rest Delhi', ['JHUNJHUNU','JAISALMER','HISSAR','HOSHIARPUR','PATIALA','CHANDIGARH','BAHADURGARH','JHAJJAR','PILIBHIT','GWALIOR','HARIDWAR','JODHPUR','JAIPUR','KOTA','AMROHA','UDAIPUR','AMRITSAR','DEHRADUN','BALLABHGARH','LUDHIANA','JALANDHAR','ROPAR', 'KAPURTHALA', 'RAMNAGAR', 'ZIRAKPUR', 'PHAGWARA', 'RAMPUR', 'NABHA', 'JAGADHRI', 'SIRHIND', 'UNNAO', 'SIWAN', 'RAJPURA', 'MOGA', 'KURUKSHETRA', 'DEHRADUN', 'ROHTAK', 'KANPUR NAGAR', 'AJMER', 'ALWAR', 'BAHADURGARH', 'CHANDIGARH','JHANSI', 'NAGAUR','SRINAGAR']),
    ('Delhi NCR',['GORAKHPUR','RUDRAPUR','AMROHA','JIND','Ghaziabad VDEL','GURGAON','NOIDA','DELHI','GHAZIABAD','FARIDABAD','NOIDA','HAPUR','GWALIOR']),
    ('Nagpur', ['JABALPUR','RATLAM','NAGPUR','JALGAON','SURGUJA','AKOLA','NAGPUR', 'RAIPUR',  'AMRAVATI', 'SATNA', 'WARDHA', 'KHANDWA', 'DURG', 'HATHNOOR', 'RAIGARH', 'RAJNANDGAON', 'GADCHIROLI', 'GONDIA', 'BEED']),
    ('Indore',['BHOPAL','INDORE','UJJAIN','RATLAM','SATNA','KHANDWA','CHHINDWARA']),
    ('Pune', ['LATUR','Parli','NASHIK','KOLHAPUR','NORTH GOA','AURANGABAD','NASHIK','NORTH GOA','VERNA','GOA', 'PANJIM', 'PUNE', 'NANDED', 'AHMEDNAGAR', 'PIMPRI', 'Daund']),
    ('Lucknow', ['FATEHPUR','SAHARANPUR','HALDWANI','BAREILLY','KANPUR NAGAR','LUCKNOW', 'SULTANPUR', 'SITAPUR', 'RAI BAREILLY', 'FAIZABAD', 'FARRUKHABAD', 'MEERUT', 'AKBARPUR', 'MAINPURI', 'SALEMPUR', 'BALLIA', 'MAHARAJGANJ', 'ORAI']),
    ('Mumbai', ['PALGHAR', 'BHIWANDI', 'VASAI', 'THANE', 'MUMBAI',]),
    ('Ahmedabad', ['GANDHIDHAM','SURAT','Vapi','Ahmedabad','Ahmedabad_9227207320','Ahmedabad_Ahmedabad','Ahmedabad_NEW MANINAGAR','Vadodara','AHMEDABAD','KACHCHH','VALSAD','JUNAGADH','BANASKANTHA','RAJKOT']),]

# Define branch data for each city (for Booking Branch filter)
branch_origin_booking_branch = {'Ahmedabad': ['Ahmedabad','Vapi','Ahmedabad_NEW NARODA NIKOL','Ahmedabad_NEW MANINAGAR','Ahmedabad_Pirana Road (Piplaj)','Ahmedabad_Piplaj','Ahmedabad_Singarva','Ahmedabad_Ahmedabad','Ahmedabad_Isanpur','SURAT','Ahmedabad_9227207320','Ahmedabad_Maninagar','Ahmedabad_Bakrol-Indore Highway','Ahmedabad_Kathwada','Ahmedabad_KUBALTHAL','Ahmedabad_','GANDHINAGAR VAHM','Ahmedabad_Nadiad','Ahmedabad_VASTRAL','KACHCHH','Ahmedabad_KUHA, CHANDIYEL','Ahmedabad_RAMOL'],
    'Bangalore': ['Bangalore','Bangalore_Bangalore','Banglore','Banglore_Bangalore'],
    'Chennai': ['Chennai','MADURAI HUB VMAA'],
    'Delhi NCR': [ 'Delhi_Nangaloi', 'Alipur', 'Delhi_LAXMI MARKET MUNIRKA', 'NOIDA_RAJENDER PLACE','Delhi', 'NOIDA', 'Delhi_LAXMI MARKET MUNIRKA', 'Delhi_Faridabad', 'Delhi_Nangaloi','Farukh Nagar', 'Alipur', 'Delhi_DWARKA SEC-7', 'NOIDA', 'Delhi_SADAR BAZAR', 'Delhi_ dwarka , new delhi','Delhi_MOTI BAGH', 'Delhi_LAXMI MARKET MUNIRKA', 'Delhi_Faridabad', 'Delhi_Nangaloi', 'Mahipalpur','Delhi_Naraina (Delhi)', 'Delhi_Kirti Nagar', 'NOIDA_RAJENDER PLACE', 'Delhi_Delhi', 'Delhi_Gurgaon','Delhi_Chippe Wara Jama Masjid (Delhi)', 'Delhi_Karnal', 'Delhi_Hari Nagar Delhi', 'Delhi_Baljeet Nagar Shadipur','Delhi_Mayapuri (Delhi)', 'Delhi_Pashim Vihar (delhi)', 'NOIDA_Anantapur'],                        
    'Hyderabad': ['VISHAKHAPATNAM', 'Hyderabad', 'Hyderabad_Karimnagar', 'Hyderabad_Dharamavaram', 'Vijayawada','Hyderabad_Nandyal', 'Hyderabad_Kadiri', 'Hyderabad_Kurnool', 'Hyderabad_', 'Hyderabad_Jubilee Hills','Hyderabad', 'Hyderabad_Nellore', 'Hyderabad_Kurnool', 'Hyderabad_Vijayawada', 'Hyderabad_Karimnagar','Hyderabad_kalyandurg', 'Hyderabad_Kadiri', 'Hyderabad_Dharamavaram', 'Hyderabad_', 'Hyderabad_Jubilee Hills','Hyderabad', 'Vijayawada', 'Hyderabad_Vijayawada', 'Hyderabad_Kothakota', 'Hyderabad_Kadiri','Hyderabad_Nellore Venkataramapuram', 'VISHAKHAPATNAM', 'Hyderabad_CUDDAPAH BRANCH', 'Hyderabad_Karimnagar','Hyderabad_', 'Hyderabad_Kurnool', 'Hyderabad_Gudur', 'Hyderabad_Anantapur', 'Hyderabad_Jubilee Hills','Hyderabad_Nellore', 'Hyderabad_Nandyal', 'Hyderabad_Rajampet', 'Hyderabad_Rayadurgam', 'Hyderabad_kalyandurg','Hyderabad_Dharamavaram', 'Hyderabad_Ranathalam', 'Hyderabad_Hindupur', 'Hyderabad_Ameerpet', 'Hyderabad_Proddatur','Vijaywada', 'Hyderabad_Trimalagiri', 'Hyderabad_BegamPet', 'Hyderabad_Adilabad', 'Hyderabad_Hyderabad','Hyderabad_Shalibanda', 'Hyderabad_Attapur MCP', 'Hyderabad_Warangal', '_Cuddapah'],
    'Lucknow': ['Lucknow','Lucknow_Kanpur'],
    'Mumbai': ['Goregaon BR', 'Bhiwandi (HUB)', 'Mumbai CHANDIVALI (HUB)_Govandi', 'Kalamboli VBHWD','Mumbai CHANDIVALI (HUB)_Khar (W)', 'Mumbai CHANDIVALI (HUB)', 'Bhiwandi (HUB)_Kalyan', 'Vasai VBHWD','Mumbai CHANDIVALI (HUB)_Kanjurmarg', 'Mumbai CHANDIVALI (HUB)_CBD Belapur', 'Mumbai CHANDIVALI (HUB)_Masjid Bunder East','Vashi VBHWD', 'Mumbai CHANDIVALI (HUB)_Chembur', 'Bhiwandi (HUB)_Bhiwandi Kudus', 'Bhivandi (HUB)','Mumbai CHANDIVALI (HUB)_Tilaknagar', 'Bhiwandi (HUB)_Tilaknagar', 'Mumbai CHANDIVALI (HUB)_Bandra West','Mumbai CHANDIVALI (HUB)_Miraroad, Andheri', 'Mumbai CHANDIVALI (HUB)_Mahalaxmi', 'Bhiwandi (HUB)_Navi Mumbai','Mumbai GOREGAON', 'Bhiwandi (HUB)_Thane'],
    'Nagpur': ['Nagpur','Nagpur_Nagpur','Nagpur_Bhandara '],
    'Indore': ['Indore VNGP'],                           
    'Pune': ['Pune VADGAON_Katraj', 'Pune VADGAON', 'Pune VADGAON_Vishrantwadi', 'Pune WADAKI','Pune VADGAON_Wagholi', 'Pune VADGAON_Sant tukaram Nagar', 'Pune VADGAON_Thergaon','Pune VADGAON_Pimpri', 'Pune VADGAON_Wanorie', 'Pune VADGAON_Thergaon', 'Pune VADGAON_Hinjewadi Phase 1','Pune VADGAON_MANJARI'],
    'Kolkata' :['Kolkata','Kolkata_Kolkata','BHUBANESHWAR VKOL','Kolkata_SALT LAKE CITY ']
}

# Create a dictionary to map cities to their corresponding last scan branches
cities_last_scan_branch = {
    'Hyderabad': ['Hyderabad_BegamPet','Hyderabad_BegamPet','Hyderabad_Hyderabad','Hyderabad_Siripur-Khagaj Nagar','Hyderabad_Kothakota','Hyderabad_Huzur Nagar','Hyderabad_Kadiri','Hyderabad_Hindupur','Hyderabad_Kakinada','Hyderabad_nalgonda','Hyderabad_CUDDAPAH BRANCH','Hyderabad_Tadipatri MAIN BRANCH','Hyderabad_Siripur-Khagaj Nagar','Hyderabad_Kothakota','Hyderabad_Huzur Nagar','Hyderabad_Kadiri','Hyderabad_Hindupur','Hyderabad_Kakinada','Hyderabad_nalgonda','Hyderabad_Guntur','Ongole VHYD','Tirupathi VHYD','Warangal VHYD','Hyderabad_Kurnool', 'Hyderabad_Ranathalam', 'Hyderabad', 'Hyderabad_Vizianagram', 'VISHAKHAPATNAM', 'Vijayawada', 'Hyderabad_', 'Ananthapur VHYD', 'Hyderabad_Kakinada', 'Rajamundhry VHYD', 'Hyderabad_Tirupathi', 'Hyderabad_Nellore Venkataramapuram', 'Hyderabad_Karimnagar', 'Hyderabad_Kadiri', 'Hyderabad_Nellore', 'Hyderabad_Hindupur', 'Hyderabad_Kakinada', 'Hyderabad_nalgonda', 'Hyderabad_Guntur', 'Guntur VHYD', 'Hyderabad_Badvel', 'Hyderabad_Khammam Grean Markt Road', 'Hyderabad_Nizamabad', 'Hyderabad_Adilabad', 'Hyderabad_Warangal karimabad', 'Hyderabad_Hyderabad', 'Hyderabad_Mancherial', 'Kakinada VHYD', 'Hyderabad_Rajamundry Thadithota', 'Hyderabad_CUDDAPAH', 'Hyderabad_Warangal', 'Cuddapah VHYD', 'Khammam VHYD', 'Srikakulam VHYD', 'Rajamundhry VHYD', 'Nizamabad VHYD', 'Nellore VHYD', 'Vizianagaram VHYD', 'Ananthapur VHYD', 'VISHAKHAPATNAM', 'Vijayawada','Kurnool VHYD','Hyderabad','Mahabub Nagar VHYD','Hyderabad_Vizianagram','Hyderabad_Kurnool','Hyderabad_Vijayawada','Adilabad VHYD','Hyderabad_kalyandurg','Hyderabad_Dharamavaram','Hyderabad_Jubilee Hills','Kurnool VHYD','Hyderabad','Mahabub Nagar VHYD','Hyderabad_Vizianagram','Hyderabad_Kurnool','Hyderabad_Vijayawada','Adilabad VHYD','Hyderabad_kalyandurg','Hyderabad_Dharamavaram','Hyderabad_Jubilee Hills',],
    'Bangalore': ['MANGALORE VBLR','BIDAR VBLR','BAGALKOT VBLR','MYSORE VBLR','BELGAVI VBLR','VIJAYAPURA VBLR','PALAGHAT VBLR','DAVENGERE VBLR','TUMKUR VBLR','TRIVANDRUM VBLR','KANNUR VBLR','HUBLI VBLR','CHITRADURGA VBLR','Bangalore', 'COCHIN VBLR', 'MYSORE VBLR', 'TRIVANDRUM VBLR', 'MANGALORE VBLR', 'KALABURGI VBLR', 'DAVENGERE VBLR', 'SHIMOGA VBLR', 'CALICUT VBLR', 'BELGAVI VBLR', 'HUBLI VBLR', 'TRICHUR VBLR', 'BELLARY VBLR', 'HOSPET VBLR'],
    'Kolkata': ['CHACHAR VKOL','JAJPUR VKOL','JHARKHAND VKOL','BHUBANESHWAR VKOL', 'SILIGURI VKOL', 'RANCHI VKOL', 'Kolkata', 'GUWAHATI VKOL', 'PATNA VKOL'],
    'Chennai': ['KRISHNAGIRI VMAA','KARUR VMAA','DINDIGUL VMAA','TIRUPPUR VMAA','TRICHY HUB VMAA','Chennai', 'SALEM HUB VMAA', 'COIMBATORE VMAA', 'MADURAI HUB VMAA', 'KANCHIPURAM VMAA', 'CHENGALPATTU VMAA', 'PONDICHERRY VMAA', 'ERODE VMAA', 'VELLORE VMAA', 'TIRUPPUR VMAA', 'TIRUNELVELI HUB VMAA'],
    'Delhi NCR': ['Farukh Nagar','Delhi_LAXMI MARKET MUNIRKA','NOIDA','Azamgarh VDEL','Alipur','Mahipalpur','Delhi_SADAR BAZAR','Ghaziabagd VDEL', 'Ambala VDEL', 'Ludhiana VDEL', 'Jammu VDEL', 'Rohtak VDEL','Allahabad VDEL',  'Delhi'],
    'Rest Delhi':['Delhi_Sindhi Colony Bani Park','Delhi_Chandigarh'],
    'Nagpur': [ 'Attara VNGP','Ambikapur VNGP','Balod VNGP','Bhanpuri VNGP','Bhatapara VNGP','Bhilai VNGP','Bilaspur VNGP','Bilha VNGP','Birgaon VNGP','Bodri VNGP','Champa VNGP','Dhamtari VNGP','Durg VNGP','Jagdalpur VNGP','Janjgir VNGP','Korba VNGP','Mahasamund VNGP','Raigarh VNGP','Rajnandgaon VNGP','Sahadol VNGP','Nagpur',  'Raipur VNGP', 'Jabalpur VNGP','Raigarh VNGP', 'Chhindwara VNGP'],
    'Indore': ['Ujjain VNGP','Khandwa VNGP','Indore VNGP','Bhopal VNGP'],
    'Lucknow': ['Varanasi VDEL','Gorakhpur VDEL','Lucknow', 'Lucknow_Kanpur'],
    'Ahmedabad': ['BANASKANTHA VAHM','GODHARA VAHM','JUNAGADH VAHM','NADIAD VAHM','PORBANDAR VAHM','SABARKANTHA VAHM','SURENDRANAGAR VAHM','SURAT','Vapi','VADODARA VAHM','GANDHIDHAM-KUTCH VAHM','Ahmedabad_9227207320','Ahmedabad_NEW MANINAGAR','Ahmedabad_Isanpur','Ahmedabad','VADODARA VAHM','GANDHIDHAM-KUTCH VAHM','Ahmedabad_9227207320','Ahmedabad_NEW MANINAGAR','Ahmedabad_Isanpur','Ahmedabad', 'ANKLESHWAR VAHM', 'VADODARA VAHM', 'RAJKOT VAHM', 'GANDHINAGAR VAHM', 'MEHSANA VAHM', 'DAHEJ VAHM', 'JAMNAGAR VAHM', 'BHUJ VAHM'],
    'Pune': ['Goa VBHWD','Pune VADGAON','Pune VADGAON_Aurangabad','Pune WADAKI','Satara VPNQV','Karad VPNQV','Sangali VPNQV','Baramati VPNQV','Daund VPNQV','Kolhapur VPNQV','Ahamad Nagar VPNQV','Kurkumbh VPNQV''Pune VADGAON'],
    'Mumbai': ['Mumbai CHANDIVALI (HUB)','Mumbai CHANDIVALI (HUB)_Govandi','Thane VBHWD','Kalamboli VBHWD','Goregaon BR','Bhiwandi (HUB)','Vasai VBHWD','Vashi VBHWD','Mumbai CHANDIVALI (HUB)_CBD Belapur']
}

# Filter the DataFrame based on 'Current Status' for Destination filter
df_destination = df[df['Current Status'].isin(allowed_statuses_destination)]

# Filter the DataFrame based on 'Current Status' for Booking Branch filter
df_booking_branch = df1[df1['Current Status'].isin(allowed_statuses_booking_branch)]

# Filter the DataFrame based on the new 'Current Status' values
df_additional_status = df2[df2['Current Status'].isin(allowed_statuses_last_scan)]

# Create variables to store DataFrames for each branch (Destination filter)
branch_dataframes_destination = {}

for city, branches in branch_origin_destination:
    filtered_df1 = df_destination[df_destination['Destination'].isin(branches)]
    branch_dataframes_destination[city] = filtered_df1

# Create variables to store DataFrames for each branch (Booking Branch filter)
branch_dataframes_booking_branch = {}

for city, branches in branch_origin_booking_branch.items():
    filtered_df2 = df_booking_branch[df_booking_branch['Booking Branch'].isin(branches)]
    branch_dataframes_booking_branch[city] = filtered_df2

# You can now access the DataFrames for each branch based on the respective filter.

# Create a dictionary to store combined DataFrames for each city Iterate through the cities and apply the filter for the last scan branch
branch_dataframes_lastscan_branch = {}

for city, last_scan_branches in cities_last_scan_branch.items():
    # Filter based on the last scan branch list
    filtered_df3 = df_additional_status[df_additional_status['Last Scan Branch'].isin(last_scan_branches)]
    branch_dataframes_lastscan_branch[city] = filtered_df3

# Create a dictionary to store combined DataFrames for each city, including the last scan branch
combined_branch_dataframes_with_lastscan = {}

for city, branches in branch_origin_destination:
    destination_df = branch_dataframes_destination.get(city)
    booking_branch_df = branch_dataframes_booking_branch.get(city)
    lastscan_branch_df = branch_dataframes_lastscan_branch.get(city)

    # Combine DataFrames even if there's no data
    combined_df = pd.concat([destination_df, booking_branch_df, lastscan_branch_df], ignore_index=True)
    
   # Check if the combined DataFrame is empty
    if not combined_df.empty:
        # Sort the combined DataFrame by "Ageing" in descending order
        combined_df = combined_df.sort_values(by='Last scan Date & time',ascending=True)
        combined_branch_dataframes_with_lastscan[city] = combined_df
    else:
        print(f"No data for {city}")
        
# Now, combined_branch_dataframes_with_lastscan contains the combined DataFrames for each branch, including the last scan branch.
# Define the directory where you want to save the Excel files

output_directory = 'output_excel_files'

# Create the directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
combined_branch_dataframes_with_lastscan[city] = combined_df


# Define the border style
border_style = Border(left=Side(style='thin'), 
                      right=Side(style='thin'), 
                      top=Side(style='thin'), 
                      bottom=Side(style='thin'))

# Define the font style for column names
column_name_font = Font(color="000000", bold=True)  # Black text color

# Define the fill color for column names
column_name_fill = PatternFill(start_color="B8CCE4", end_color="B8CCE4", fill_type="solid")

# Iterate through the combined DataFrames for each branch
for branch_name, combined_df in combined_branch_dataframes_with_lastscan.items():
    # Define the Excel file name for this branch
    excel_file_name = os.path.join(output_directory, f'{branch_name}_NO_INFORMATION.xlsx')
    
    # Create a new Excel workbook
    wb = openpyxl.Workbook()
    
    # Create a new worksheet
    ws = wb.active
    ws.title = "NO INFORMATION"
    
    # Convert the DataFrame to rows and add them to the worksheet
    for r_idx, row in enumerate(dataframe_to_rows(combined_df, index=False, header=True), 1):
        for c_idx, value in enumerate(row, 1):
            cell = ws.cell(row=r_idx, column=c_idx, value=value)
            
            # Center the data in the cell
            cell.alignment = Alignment(horizontal='center', vertical='center')
            
            # Apply the border style to the cell
            cell.border = border_style
    
    # Apply styling to the header row (column names)
    for cell in ws[1]:
        cell.font = column_name_font
        cell.fill = column_name_fill
    
    # Automatically adjust column widths based on content
    for column in ws.columns:
        max_length = 0
        column = [cell for cell in column]
        for cell in column:
            try:  # Necessary to avoid error on empty cells
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass
        adjusted_width = (max_length + 2)
        ws.column_dimensions[column[0].column_letter].width = adjusted_width
        # Define the fill styles for highlighting (red fill and orange fill)
        red_fill = PatternFill(start_color='FFFF0000', end_color='FFFF0000', fill_type='solid')
        orange_fill = PatternFill(start_color='FFFFA500', end_color='FFFFA500', fill_type='solid')

        # Iterate through each row and apply conditional formatting based on Ageing
        for row in ws.iter_rows(min_row=2, max_col=ws.max_column, max_row=ws.max_row):
            if row[2].value is not None:
                ageing = row[2].value
                if ageing >= 10:
                    fill_style = red_fill
                elif 5 <= ageing <= 9:
                    fill_style = orange_fill
                else:
                    fill_style = None
                
                if fill_style is not None:
                    for cell in row:
                        cell.fill = fill_style
           
    
    # Save the Excel file
    wb.save(excel_file_name)

print("Excel files saved successfully.")


No data for Nagpur
Excel files saved successfully.


In [10]:
# Directory where the Excel files are located
excel_files_directory = 'output_excel_files'
 
# Initialize a dictionary to store the counts for each city and month
city_month_counts = {}

# List of months for which you want to count data
months = ['September', 'October','November']

# Iterate through the Excel files in the directory
for filename in os.listdir(excel_files_directory):
    if filename.endswith('.xlsx'):
        # Extract the city name from the filename (assuming the filename format is 'CityName_NO_INFORMATION.xlsx')
        city = filename.split('_')[0]
        
#         print(f"Processing file for city: {city}")

        # Read the Excel file into a DataFrame
        excel_file_path = os.path.join(excel_files_directory, filename)
        df = pd.read_excel(excel_file_path)

        # Check if the 'Booking Date' column exists in the DataFrame
        if 'Booking Date' not in df.columns:
            # If not, print a message and skip this file
            print("No 'Booking Date' column found in this file. Skipping...")
            continue

        # Specify the correct date format
        date_format = "%d-%m-%Y"

        # Convert the 'Booking Date' column to datetime using the correct format
        df['Booking Date'] = pd.to_datetime(df['Booking Date'], format=date_format)


        # Initialize counts for each month
        month_counts = {month: 0 for month in months}

        # Count the data for each month
        for month in months:
            # Extract the month and year using the correct column name
            booking_date_col = 'Booking Date'  # Default column name
            if 'Booking Date' not in df.columns:
                # Modify this part based on the actual column name in your file
                booking_date_col = 'The Correct Column Name'

            df[booking_date_col] = pd.to_datetime(df[booking_date_col])
            df_month = df[df[booking_date_col].dt.month == months.index(month) + 9]
            month_counts[month] = len(df_month)

        # Add the counts to the dictionary
        city_month_counts[city] = month_counts

# Create a DataFrame from the counts dictionary
city_month_counts_df= pd.DataFrame(city_month_counts).transpose()
city_month_counts_df.columns = months
# Add a new column containing the sum for each city
city_month_counts_df['Total'] = city_month_counts_df.sum(axis=1)

city_month_counts_df.to_excel(f'History/Summary_{current_date}.xlsx')

# Display the resulting DataFrame
city_month_counts_df


,September,October,November,Total
Ahmedabad,0,0,0,0
Bangalore,0,1,14,15
Chennai,0,1,10,11
Delhi NCR,0,1,2,3
Hyderabad,0,4,13,17
Indore,0,0,0,0
Kolkata,0,2,8,10
Lucknow,0,0,0,0
Mumbai,0,0,5,5
Nagpur,0,0,0,0


In [11]:
city_month_counts_df.to_excel(f'No Information_{current_date}.xlsx')

In [17]:
# Define recipient and CC email addresses, and email template for each city
email_info = {
    'Hyderabad': {
        'subject': "HYDERABAD NO INFORMATION UPDATION",
        'to_email': ['dileep.s@boxnfreight.com','priya.z@boxnfreight.com', 'ops.hyderabad@boxnfreight.com', 'ops.vijaywada@boxnfreight.com', 'ops.vizag@boxnfreight.com'],
        'cc_email': ['chand.s@boxnfreight.com'],
        'body': """\
            <p>Dear Dileep Ji and Priya Ma’am,</p>
            <p>Kindly find the attached file of No Information Shipment’s. Kindly Update the Delivery details of the same. </p>
            {html_table}


            <p style="color: #3366cc; font-size: 18px; font-weight: bold;">Best Regards,</p>
            <p style="color: #3366cc; font-size: 16px; font-weight: bold;">Sanil Kumar Barik</p>
            <p style="color: #C45911; font-size: 16px; font-weight: bold;">Executive Operations Analyst</p>
            <p style="color: #3366cc; font-size: 16px; font-weight: bold;">+91 9987763071</p>
            <p style="color: #990099; font-size: 16px; font-weight: bold;">opscoordinator@boxnfreight.com</p>
            <p style="color: #1F4E79; font-size: 16px; font-weight: bold;">BOX <span style="color:#C45911">N</span> FREIGHT LOGISTICS SOLUTION PVT LTD</p>
            <p style="color: #0070C0; font-size: 16px; font-weight: bold;">133 &amp; 134, Udyog Bhavan, Sonawala Road,</p>
            <p style="color: #0070C0; font-size: 16px; font-weight: bold;">Goregaon East, Mumbai – 400063</p>
        """
    },
    
    'Bangalore': {
        'subject': "BANGALORE NO INFORMATION UPDATION",
        'to_email': ['venkatesh.k@boxnfreight.com', 'shiva.k@boxnfreight.com', 'ops.bengaluru@boxnfreight.com'],
        'cc_email': ['chand.s@boxnfreight.com'],
        'body': """\
            <p>Dear Venkatesh Ji,</p>
            <p>Kindly find the attached file of No Information Shipment’s. Kindly Update the Delivery details of the same. </p>
            {html_table}


            <p style="color: #3366cc; font-size: 18px; font-weight: bold;">Best Regards,</p>
            <p style="color: #3366cc; font-size: 16px; font-weight: bold;">Sanil Kumar Barik</p>
            <p style="color: #C45911; font-size: 16px; font-weight: bold;">Executive Operations Analyst</p>
            <p style="color: #3366cc; font-size: 16px; font-weight: bold;">+91 9987763071</p>
            <p style="color: #990099; font-size: 16px; font-weight: bold;">opscoordinator@boxnfreight.com</p>
            <p style="color: #1F4E79; font-size: 16px; font-weight: bold;">BOX <span style="color:#C45911">N</span> FREIGHT LOGISTICS SOLUTION PVT LTD</p>
            <p style="color: #0070C0; font-size: 16px; font-weight: bold;">133 &amp; 134, Udyog Bhavan, Sonawala Road,</p>
            <p style="color: #0070C0; font-size: 16px; font-weight: bold;">Goregaon East, Mumbai – 400063</p>
        """
    },
    
    'Kolkata': {
        'subject': "KOLKATA NO INFORMATION UPDATION",
        'to_email': ['sandip@boxnfreight.com', 'ops.Kolkata@boxnfreight.com'],
        'cc_email': ['chand.s@boxnfreight.com'],
        'body': """\
            <p>Dear Sandip Ji,</p>
            <p>Kindly find the attached file of No Information Shipment’s. Kindly Update the Delivery details of the same. </p>
            {html_table}


            <p style="color: #3366cc; font-size: 18px; font-weight: bold;">Best Regards,</p>
            <p style="color: #3366cc; font-size: 16px; font-weight: bold;">Sanil Kumar Barik</p>
            <p style="color: #C45911; font-size: 16px; font-weight: bold;">Executive Operations Analyst</p>
            <p style="color: #3366cc; font-size: 16px; font-weight: bold;">+91 9987763071</p>
            <p style="color: #990099; font-size: 16px; font-weight: bold;">opscoordinator@boxnfreight.com</p>
            <p style="color: #1F4E79; font-size: 16px; font-weight: bold;">BOX <span style="color:#C45911">N</span> FREIGHT LOGISTICS SOLUTION PVT LTD</p>
            <p style="color: #0070C0; font-size: 16px; font-weight: bold;">133 &amp; 134, Udyog Bhavan, Sonawala Road,</p>
            <p style="color: #0070C0; font-size: 16px; font-weight: bold;">Goregaon East, Mumbai – 400063</p>
        """
    },
    
    'Chennai': {
        'subject': "CHENNAI NO INFORMATION UPDATION",
        'to_email': ['vimal.k@boxnfreight.com', 'ops.chennai@boxnfreight.com','singaravelu.t@boxnfreight.com'],
        'cc_email': ['chand.s@boxnfreight.com'],
        'body': """\
            <p>Dear Vimal Ji,</p>
            <p>Kindly find the attached file of No Information Shipment’s. Kindly Update the Delivery details of the same. </p>
            {html_table}


            <p style="color: #3366cc; font-size: 18px; font-weight: bold;">Best Regards,</p>
            <p style="color: #3366cc; font-size: 16px; font-weight: bold;">Sanil Kumar Barik</p>
            <p style="color: #C45911; font-size: 16px; font-weight: bold;">Executive Operations Analyst</p>
            <p style="color: #3366cc; font-size: 16px; font-weight: bold;">+91 9987763071</p>
            <p style="color: #990099; font-size: 16px; font-weight: bold;">opscoordinator@boxnfreight.com</p>
            <p style="color: #1F4E79; font-size: 16px; font-weight: bold;">BOX <span style="color:#C45911">N</span> FREIGHT LOGISTICS SOLUTION PVT LTD</p>
            <p style="color: #0070C0; font-size: 16px; font-weight: bold;">133 &amp; 134, Udyog Bhavan, Sonawala Road,</p>
            <p style="color: #0070C0; font-size: 16px; font-weight: bold;">Goregaon East, Mumbai – 400063</p>
        """
    },
    
    'Rest Delhi': {
        'subject': "Rest Delhi NO INFORMATION UPDATION",
        'to_email': ['mohit.t@boxnfreight.com'],
        'cc_email': ['chand.s@boxnfreight.com','amitesh.k@boxnfreight.com'],
        'body': """\
            <p>Dear Mohit Ji,</p>
            <p>Kindly find the attached file of No Information Shipment’s. Kindly Update the Delivery details of the same. </p>
            {html_table}


            <p style="color: #3366cc; font-size: 18px; font-weight: bold;">Best Regards,</p>
            <p style="color: #3366cc; font-size: 16px; font-weight: bold;">Sanil Kumar Barik</p>
            <p style="color: #C45911; font-size: 16px; font-weight: bold;">Executive Operations Analyst</p>
            <p style="color: #3366cc; font-size: 16px; font-weight: bold;">+91 9987763071</p>
            <p style="color: #990099; font-size: 16px; font-weight: bold;">opscoordinator@boxnfreight.com</p>
            <p style="color: #1F4E79; font-size: 16px; font-weight: bold;">BOX <span style="color:#C45911">N</span> FREIGHT LOGISTICS SOLUTION PVT LTD</p>
            <p style="color: #0070C0; font-size: 16px; font-weight: bold;">133 &amp; 134, Udyog Bhavan, Sonawala Road,</p>
            <p style="color: #0070C0; font-size: 16px; font-weight: bold;">Goregaon East, Mumbai – 400063</p>
        """
    },
    
    'Delhi NCR': {
        'subject': "DELHI NO INFORMATION UPDATION",
        'to_email': ['bhupendra.s@boxnfreight.com'],
        'cc_email': ['chand.s@boxnfreight.com','amitesh.k@boxnfreight.com'],
        'body': """\
            <p>Dear Bhupendra Ji,</p>
            <p>Kindly find the attached file of No Information Shipment’s. Kindly Update the Delivery details of the same. </p>
            {html_table}
    

            <p style="color: #3366cc; font-size: 18px; font-weight: bold;">Best Regards,</p>
            <p style="color: #3366cc; font-size: 16px; font-weight: bold;">Sanil Kumar Barik</p>
            <p style="color: #C45911; font-size: 16px; font-weight: bold;">Executive Operations Analyst</p>
            <p style="color: #3366cc; font-size: 16px; font-weight: bold;">+91 9987763071</p>
            <p style="color: #990099; font-size: 16px; font-weight: bold;">opscoordinator@boxnfreight.com</p>
            <p style="color: #1F4E79; font-size: 16px; font-weight: bold;">BOX <span style="color:#C45911">N</span> FREIGHT LOGISTICS SOLUTION PVT LTD</p>
            <p style="color: #0070C0; font-size: 16px; font-weight: bold;">133 &amp; 134, Udyog Bhavan, Sonawala Road,</p>
            <p style="color: #0070C0; font-size: 16px; font-weight: bold;">Goregaon East, Mumbai – 400063</p>
        """
    },
    
    'Nagpur': {
        'subject': "NAGPUR NO INFORMATION UPDATION",
        'to_email': ['ops.nagpur@boxnfreight.com','abhishek.k@boxnfreight.com', 'nilesh.g@boxnfreight.com'],
        'cc_email': ['chand.s@boxnfreight.com','hanif.s@boxnfreight.com'],
        'body': """\
            <p>Dear Nilesh Ji,</p>
            <p>Kindly find the attached file of No Information Shipment’s. Kindly Update the Delivery details of the same. </p>
            {html_table}


            <p style="color: #3366cc; font-size: 18px; font-weight: bold;">Best Regards,</p>
            <p style="color: #3366cc; font-size: 16px; font-weight: bold;">Sanil Kumar Barik</p>
            <p style="color: #C45911; font-size: 16px; font-weight: bold;">Executive Operations Analyst</p>
            <p style="color: #3366cc; font-size: 16px; font-weight: bold;">+91 9987763071</p>
            <p style="color: #990099; font-size: 16px; font-weight: bold;">opscoordinator@boxnfreight.com</p>
            <p style="color: #1F4E79; font-size: 16px; font-weight: bold;">BOX <span style="color:#C45911">N</span> FREIGHT LOGISTICS SOLUTION PVT LTD</p>
            <p style="color: #0070C0; font-size: 16px; font-weight: bold;">133 &amp; 134, Udyog Bhavan, Sonawala Road,</p>
            <p style="color: #0070C0; font-size: 16px; font-weight: bold;">Goregaon East, Mumbai – 400063</p>
        """
    },
    
    'Lucknow': {
        'subject': "LUCKNOW NO INFORMATION UPDATION",
        'to_email': ['ops.lucknow@boxnfreight.com', 'manoj.g@boxnfreight.com'],
        'cc_email': ['chand.s@boxnfreight.com'],
        'body': """\
            <p>Dear Manoj Ji,</p>
            <p>Kindly find the attached file of No Information Shipment’s. Kindly Update the Delivery details of the same. </p>
            {html_table}


            <p style="color: #3366cc; font-size: 18px; font-weight: bold;">Best Regards,</p>
            <p style="color: #3366cc; font-size: 16px; font-weight: bold;">Sanil Kumar Barik</p>
            <p style="color: #C45911; font-size: 16px; font-weight: bold;">Executive Operations Analyst</p>
            <p style="color: #3366cc; font-size: 16px; font-weight: bold;">+91 9987763071</p>
            <p style="color: #990099; font-size: 16px; font-weight: bold;">opscoordinator@boxnfreight.com</p>
            <p style="color: #1F4E79; font-size: 16px; font-weight: bold;">BOX <span style="color:#C45911">N</span> FREIGHT LOGISTICS SOLUTION PVT LTD</p>
            <p style="color: #0070C0; font-size: 16px; font-weight: bold;">133 &amp; 134, Udyog Bhavan, Sonawala Road,</p>
            <p style="color: #0070C0; font-size: 16px; font-weight: bold;">Goregaon East, Mumbai – 400063</p> 
        """
    },
    
    'Ahmedabad': {
        'subject': "AHMEDABAD NO INFORMATION UPDATION",
        'to_email': ['harish.s@boxnfreight.com', 'anil.p@boxnfreight.com'],
        'cc_email': ['chand.s@boxnfreight.com'],
        'body': """\
            <p>Dear Harish Ji/ Anil Ji,</p>
            <p>Kindly find the attached file of No Information Shipment’s. Kindly Update the Delivery details of the same. </p>
            {html_table}
    

            <p style="color: #3366cc; font-size: 18px; font-weight: bold;">Best Regards,</p>
            <p style="color: #3366cc; font-size: 16px; font-weight: bold;">Sanil Kumar Barik</p>
            <p style="color: #C45911; font-size: 16px; font-weight: bold;">Executive Operations Analyst</p>
            <p style="color: #3366cc; font-size: 16px; font-weight: bold;">+91 9987763071</p>
            <p style="color: #990099; font-size: 16px; font-weight: bold;">opscoordinator@boxnfreight.com</p>
            <p style="color: #1F4E79; font-size: 16px; font-weight: bold;">BOX <span style="color:#C45911">N</span> FREIGHT LOGISTICS SOLUTION PVT LTD</p>
            <p style="color: #0070C0; font-size: 16px; font-weight: bold;">133 &amp; 134, Udyog Bhavan, Sonawala Road,</p>
            <p style="color: #0070C0; font-size: 16px; font-weight: bold;">Goregaon East, Mumbai – 400063</p>
        """
    },
    
    'Pune': {
        'subject': "PUNE NO INFORMATION UPDATION",
        'to_email': ['vaibhav.s@boxnfreight.com', 'ops.punewadgaon@boxnfreight.com'],
        'cc_email': ['chand.s@boxnfreight.com','hanif.s@boxnfreight.com'],
        'body': """\
            <p>Dear Team,</p>
            <p>Kindly find the attached file of No Information Shipment’s. Kindly Update the Delivery details of the same. </p>
            {html_table}
    

            <p style="color: #3366cc; font-size: 18px; font-weight: bold;">Best Regards,</p>
            <p style="color: #3366cc; font-size: 16px; font-weight: bold;">Sanil Kumar Barik</p>
            <p style="color: #C45911; font-size: 16px; font-weight: bold;">Executive Operations Analyst</p>
            <p style="color: #3366cc; font-size: 16px; font-weight: bold;">+91 9987763071</p>
            <p style="color: #990099; font-size: 16px; font-weight: bold;">opscoordinator@boxnfreight.com</p>
            <p style="color: #1F4E79; font-size: 16px; font-weight: bold;">BOX <span style="color:#C45911">N</span> FREIGHT LOGISTICS SOLUTION PVT LTD</p>
            <p style="color: #0070C0; font-size: 16px; font-weight: bold;">133 &amp; 134, Udyog Bhavan, Sonawala Road,</p>
            <p style="color: #0070C0; font-size: 16px; font-weight: bold;">Goregaon East, Mumbai – 400063</p>
        """
    },

      'Mumbai': {
        'subject': "MUMBAI NO INFORMATION UPDATION",
        'to_email': ['vishal.k@boxnfreight.com', 'ops.bhiwandi@boxnfreight.com','tushar.s@boxnfreight.com'],
        'cc_email': ['chand.s@boxnfreight.com','hanif.s@boxnfreight.com'],
        'body': """\
            <p>Dear Vishal Ji/Tushar Ji,</p>
            <p>Kindly find the attached file of No Information Shipment’s. Kindly Update the Delivery details of the same. </p>
            {html_table}


            <p style="color: #3366cc; font-size: 18px; font-weight: bold;">Best Regards,</p>
            <p style="color: #3366cc; font-size: 16px; font-weight: bold;">Sanil Kumar Barik</p>
            <p style="color: #C45911; font-size: 16px; font-weight: bold;">Executive Operations Analyst</p>
            <p style="color: #3366cc; font-size: 16px; font-weight: bold;">+91 9987763071</p>
            <p style="color: #990099; font-size: 16px; font-weight: bold;">opscoordinator@boxnfreight.com</p>
            <p style="color: #1F4E79; font-size: 16px; font-weight: bold;">BOX <span style="color:#C45911">N</span> FREIGHT LOGISTICS SOLUTION PVT LTD</p>
            <p style="color: #0070C0; font-size: 16px; font-weight: bold;">133 &amp; 134, Udyog Bhavan, Sonawala Road,</p>
            <p style="color: #0070C0; font-size: 16px; font-weight: bold;">Goregaon East, Mumbai – 400063</p>
        """
    }
}

# Function to send email using Outlook
def send_email(to_email, cc_email, subject, message):
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = ';'.join(to_email)
    mail.CC = ';'.join(cc_email)
    mail.Subject = subject
    mail.HTMLBody = message
    mail.Send()

# Send region-wise emails
for region_name, region_data in combined_branch_dataframes_with_lastscan.items():
    if not region_data.empty:  # Check if the DataFrame is not empty
        email_details = email_info.get(region_name)
        if email_details:
            # Convert the DataFrame to an HTML table with styling
            html_table = region_data.to_html(classes='table table-bordered table-striped', escape=False, index=False)

            # Apply styling to the HTML table (column headers in light blue)
            styled_html_table = html_table.replace('<thead>', '<thead style="background-color: lightblue;">')

            # Get email details
            subject = email_details['subject']
            to_email = email_details['to_email']
            cc_email = email_details['cc_email']
            body = email_details['body'].format(current_date=current_date, html_table=styled_html_table)
            
            # Send the email
            try:
                send_email(to_email, cc_email, subject, body)
                print(f"Email sent to {', '.join(to_email)} and {', '.join(cc_email)}")
            except Exception as e:
                print(f"Error sending email to {', '.join(to_email)} and {', '.join(cc_email)}: {e}")
    else:
         print(f"No data for {region_name}. Email not sent.")

Email sent to chand.s@boxnfreight.com and chand.s@boxnfreight.com
